# Interest Rate Model Calibration


In [ ]:
using Pkg
Pkg.activate(".")

In [ ]:
using CSV
using DataFrames
using DiffFusion
using ForwardDiff
using PlotlyJS

## Reference Data

### Volatility Data

In [ ]:
file_name = "data/standard_deviation_30days.csv"
std_table = DataFrame(CSV.File(file_name))
std_table = stack(std_table, 3:size(std_table)[2])
std_table[!, "VOLATILITY"] = std_table[!, "value"] / sqrt(30.0/365.0);
std_table[!, "YEARS"] = std_table[!, "MONTHS"] / 12;

### Correlation Data

In [ ]:
file_name = "data/correlations_30days.csv"
corr_table = DataFrame(CSV.File(file_name))
corr_table = stack(corr_table, 5:size(corr_table)[2])
corr_table[!, "YEARS1"] = Int.(corr_table[!, "MONTHS1"] / 12)
corr_table[!, "YEARS2"] = Int.(corr_table[!, "MONTHS2"] / 12);

## Plotting Functions

### Plot Volatility

In [ ]:
function plot_volatility(
    std_table,
    currency,
    model_terms,
    model_values,
    scaling,
    )
    #
    table = filter(:CURRENCY => ==(currency), std_table)
    table[!,"VOLATILITY"] = table[!,"VOLATILITY"] * scaling # in percent/bp
    #
    b = box(table, x=:YEARS, y=:VOLATILITY, name="historical volatility")
    #
    s = scatter(
        x = model_terms,
        y = model_values * scaling,
        mode = "markers",
        marker = attr(size=12, line=attr(width=2, color="DarkSlateGrey")),
        name="model-implied volatility",
    )
    #
    if scaling == 100.0
        scaling_unit = " (%)"
    elseif scaling == 10000.0
        scaling_unit = " (bp)"
    else
        scaling_unit = ""
    end
    layout = Layout(
        title= currency * " Volatility",
        xaxis_title="rate term",
        yaxis_title="volatility" * scaling_unit,
        legend_title="Output",
        font=attr(
            family="Arial",
            size=12,
            color="RebeccaPurple"
        )
    )
    return plot([b, s], layout)
end

### Plot Correlations

In [ ]:
@inline function iuppert(k::Integer,n::Integer)
    i = n - 1 - floor(Int,sqrt(-8*k + 4*n*(n-1) + 1)/2 - 0.5)
    j = k + i + ( (n-i+1)*(n-i) - n*(n-1) )÷2
    return i, j
end

function make_index_list(curr_and_year::AbstractVector{<:Tuple})
    l = curr_and_year  # abbreviation
    n = length(curr_and_year)
    return [
        (l[idx[1]][1], l[idx[1]][2], l[idx[2]][1], l[idx[2]][2])
        for idx in (iuppert(k, n) for k in 1:Int(n*(n-1)/2))
    ]
end

function make_index_list(curr_and_year_1::AbstractVector{<:Tuple}, curr_and_year_2::AbstractVector{<:Tuple})
    l1 = curr_and_year_1  # abbreviation
    l2 = curr_and_year_2  # abbreviation
    return [
        (l1[1], l1[2], l2[1], l2[2])
        for l1 in curr_and_year_1 for l2 in curr_and_year_2
    ]
end

function make_sub_table(table, index_list)
    t = table  # abbreviation
    l = index_list  # abbreviation
    tmp = vcat([
        [
            t[(t.CURRENCY1 .== l[1]) .&& (t.YEARS1 .== l[2]) .&& (t.CURRENCY2 .== l[3]) .&& (t.YEARS2 .== l[4]),:],
            t[(t.CURRENCY1 .== l[3]) .&& (t.YEARS1 .== l[4]) .&& (t.CURRENCY2 .== l[1]) .&& (t.YEARS2 .== l[2]),:],
        ]
        for l in index_list
    ]...)
    return vcat(tmp...)
end

function plot_correlation(
    table,
    index_list,
    values,
    )
    #
    t = make_sub_table(table, index_list)
    t[!, "XLABEL"] = t[!, "CURRENCY1"] .* "_" .* string.(t[!, "YEARS1"]) .* "__" .* t[!, "CURRENCY2"] .* "_" .* string.(t[!, "YEARS2"])
    t[!, "CORRELATION"] = t[!, "value"] .* 100  # in percent
    b = box(t, x=:XLABEL, y=:CORRELATION, name="historical correlation")
    #
    xlabels = [
        t[1] * "_" * string.(t[2]) * "__" * t[3] * "_" .* string(t[4])
        for t in index_list
    ]
    yvalues = values .* 100 # in percent
    s = scatter(
        x = xlabels,
        y = yvalues,
        mode = "markers",
        marker = attr(size=12, line=attr(width=2, color="DarkSlateGrey")),
        name="model-implied correlation",
    )
    #
    layout = Layout(
        title= "Correlation",
        xaxis_title="risk factors",
        yaxis_title="correlation (%)",
        legend_title="Output",
        font=attr(
            family="Arial",
            size=12,
            color="RebeccaPurple"
        )
    )
    return plot([b, s], layout)
end

## Model Setup

In [ ]:
model_params = Dict([
    (("delta_1", ""), 0.01),
    (("delta_2", ""), 10.0),
    (("delta_3", ""), 20.0),
    #
    (("chi_1", ""), 0.01),
    (("chi_2", ""), 0.10),
    (("chi_3", ""), 0.15),
    #
    (("EUR_f_1", ""), 0.0070),
    (("EUR_f_2", ""), 0.0070),
    (("EUR_f_3", ""), 0.0070),
    #
    (("EUR_f_1", "EUR_f_2"), 0.1),
    (("EUR_f_2", "EUR_f_3"), 0.2),
    (("EUR_f_1", "EUR_f_3"), 0.2),
    #
]);

In [ ]:
function model(model_params)
    #
    ch = DiffFusion.correlation_holder("Std", "<>", typeof(model_params[("EUR_f_1","EUR_f_2")]))
    for t in model_params
        if t[1][2] != ""
            DiffFusion.set_correlation!(ch, t[1][1], t[1][2], t[2])
        end
    end
    #
    t0 = [ 0.0 ]
    d = model_params  # abbreviation
    #
    delta = [ d[("delta_1","")], d[("delta_2","")], d[("delta_3","")] ]
    chi = [ d[("chi_1","")], d[("chi_2","")], d[("chi_3","")] ]
    #
    hjm_eur = DiffFusion.gaussian_hjm_model(
        "EUR",
        DiffFusion.flat_parameter(delta),
        DiffFusion.flat_parameter(chi),
        DiffFusion.backward_flat_volatility("",t0, [ d[("EUR_f_1","")] d[("EUR_f_2","")] d[("EUR_f_3","")] ]' ),  # sigma
        ch,
        nothing,
    )
    #
    empty_key = DiffFusion._empty_context_key
    #
    ctx = DiffFusion.Context("Std",
        DiffFusion.NumeraireEntry("EUR", "EUR", Dict()),
        Dict{String, DiffFusion.RatesEntry}([
            ("EUR", DiffFusion.RatesEntry("EUR","EUR", Dict())),
            ("USD", DiffFusion.RatesEntry("USD", "USD", Dict())),
            ("GBP", DiffFusion.RatesEntry("GBP", "GBP", Dict())),
        ]),
        Dict{String, DiffFusion.AssetEntry}([
            ("USD-EUR", DiffFusion.AssetEntry("USD-EUR", "USD-EUR", "EUR", "USD", "pa/USD-EUR", Dict(), Dict())),
            ("GBP-EUR", DiffFusion.AssetEntry("GBP-EUR", "GBP-EUR", "EUR", "GBP", "pa/GBP-EUR", Dict(), Dict())),
        ]),
        Dict{String, DiffFusion.ForwardIndexEntry}(),
        Dict{String, DiffFusion.FutureIndexEntry}(),
        Dict{String, DiffFusion.FixingEntry}(),
    )
    #
    return (ctx, hjm_eur, ch)
end

In [ ]:
function model_correlation(index_list, keys_and_terms, corr_matrix)
    return [
        corr_matrix[
            findall(x->x==(l[1],l[2]),keys_and_terms)[begin],
            findall(x->x==(l[3],l[4]),keys_and_terms)[begin]
        ]
        for l in index_list
    ]
end

## Update and Output

In [ ]:
function update_plots!(
    param_list, # ::AbstractArray{<:Tuple},
    model_params,
    std_table,
    corr_table;
    plot_vols = false,
    plot_rates_corrs = false,
    plot_fx_corrs = false,
    plot_fx_rates_corrs = false,
    plot_rates_rates_corrs = false,
    )
    #
    for t in param_list
        model_params[(t[1], t[2])] = t[3]
    end
    #
    (ctx, mdl, ch) = model(model_params)
    keys_and_terms = [
        ("EUR", 1),
        ("EUR", 2),
        ("EUR", 5),
        ("EUR", 10),
        ("EUR", 20),
    ]
    (v, C) = DiffFusion.reference_rate_volatility_and_correlation(keys_and_terms, ctx, mdl, ch, 0.0, 1.0/12.0)
    #
    model_terms = [1.0, 2.0, 5.0, 10.0, 20.0]
    v_eur = v[1:5]
    #
    if plot_vols
        display("text/markdown", "## Volatilities:")
        display(plot_volatility(std_table, "EUR", model_terms, v_eur, 1.0e+4))
    end
    #
    rf_eur = [ ("EUR", 1), ("EUR", 2), ("EUR", 5), ("EUR", 10), ("EUR", 20),]
    #
    if plot_rates_corrs
        display("text/markdown", "## Rates Correlations:")
        idx = make_index_list(rf_eur)
        display(plot_correlation(corr_table, idx, model_correlation(idx, keys_and_terms, C)))
    end
end

## Model Analysis

In [ ]:
update_plots!(
    [
    ],
    model_params, std_table, corr_table,
    plot_vols = true,
    plot_rates_corrs = true,
)

## Sensitivity Analysis

In [ ]:
function model_outputs(X::AbstractVector)
    @assert length(X) == 12
    model_params = Dict([
        (("delta_1", ""), X[1]),
        (("delta_2", ""), X[2]),
        (("delta_3", ""), X[3]),
        #
        (("chi_1", ""), X[4]),
        (("chi_2", ""), X[5]),
        (("chi_3", ""), X[6]),
        #
        (("EUR_f_1", ""), X[7]),
        (("EUR_f_2", ""), X[8]),
        (("EUR_f_3", ""), X[9]),
        #
        (("EUR_f_1", "EUR_f_2"), X[10]),
        (("EUR_f_2", "EUR_f_3"), X[11]),
        (("EUR_f_1", "EUR_f_3"), X[12]),
    ])
    #
    (ctx, mdl, ch) = model(model_params)
    keys_and_terms = [
        ("EUR", 1),
        ("EUR", 2),
        ("EUR", 5),
        ("EUR", 10),
        ("EUR", 20),
    ]
    (v, C) = DiffFusion.reference_rate_volatility_and_correlation(keys_and_terms, ctx, mdl, ch, 0.0, 1.0/12.0)
    #
    model_terms = [1.0, 2.0, 5.0, 10.0, 20.0]
    v_eur = v[1:5]
    rf_eur = [ ("EUR", 1), ("EUR", 2), ("EUR", 5), ("EUR", 10), ("EUR", 20),]
    #
    idx = make_index_list(rf_eur)
    c_eur = model_correlation(idx, keys_and_terms, C)
    # return vcat( v_eur * 10000, c_eur * 100 )
    return vcat( v_eur, c_eur )
end

In [ ]:
x_labels = [
    "delta_1",
    "delta_2",
    "delta_3",
    #
    "chi_1",
    "chi_2",
    "chi_3",
    #
    "EUR_f_1",
    "EUR_f_2",
    "EUR_f_3",
    #
    "EUR_f_1__EUR_f_2",
    "EUR_f_2__EUR_f_3",
    "EUR_f_1__EUR_f_3",
]

rf_eur = [ ("EUR", 1), ("EUR", 2), ("EUR", 5), ("EUR", 10), ("EUR", 20),]
y_labels_vol = [
    l[1] * "_" * string(l[2])
    for l in rf_eur
]
idx = make_index_list(rf_eur)
y_labels_corr = [
    l[1] * "_" * string(l[2]) * "__" * l[3] * "_" * string(l[4])
    for l in idx
]
y_labels = vcat(y_labels_vol, y_labels_corr);

In [ ]:
X = [
    model_params[("delta_1", "")],
    model_params[("delta_2", "")],
    model_params[("delta_3", "")],
    #
    model_params[("chi_1", "")],
    model_params[("chi_2", "")],
    model_params[("chi_3", "")],
    #
    model_params[("EUR_f_1", "")],
    model_params[("EUR_f_2", "")],
    model_params[("EUR_f_3", "")],
    #
    model_params[("EUR_f_1", "EUR_f_2")],
    model_params[("EUR_f_2", "EUR_f_3")],
    model_params[("EUR_f_1", "EUR_f_3")],
];

We calculate the Jacobian using `ForwardDiff`.

In [ ]:
J = ForwardDiff.jacobian(model_outputs, X)
show(stdout, "text/plain", round.(J, digits=4))

In [ ]:
x1 = x_labels[1:3]
x2 = x_labels[4:6]
x3 = x_labels[7:9]
x4 = x_labels[10:12]
#
y1 = y_labels[1:5]
y2 = y_labels[6:15]
#
J11 = J[1:5,1:3]  * 1e+4    # bp / y
J21 = J[6:15,1:3] * 1e+2    # %  / y
J12 = J[1:5,4:6]  * 1e+2    # bp / %
J22 = J[6:15,4:6]           # %  / %
J13 = J[1:5,7:9]            # bp / bp
J23 = J[6:15,7:9] * 1e-2    # %  / bp
J14 = J[1:5,10:12] * 1e+2   # bp / %  
J24 = J[6:15,10:12]         # % / %


hm11 = heatmap(x=x1, y=y1, z=J11, showscale=false,)
hm21 = heatmap(x=x1, y=y2, z=J21, showscale=false,)
#
hm12 = heatmap(x=x2, y=y1, z=J12, showscale=false,)
hm22 = heatmap(x=x2, y=y2, z=J22, showscale=false,)
#
hm13 = heatmap(x=x3, y=y1, z=J13, showscale=false,)
hm23 = heatmap(x=x3, y=y2, z=J23, showscale=false,)
#
hm14 = heatmap(x=x4, y=y1, z=J14, showscale=false,)
hm24 = heatmap(x=x4, y=y2, z=J24, showscale=false,)

as_text(x) = string(round(x, digits=2))
#
ann11 = [
    attr(y=i-1, x=j-1, text=as_text(J11[i,j]), xref="x1", yref="y1", showarrow=false)
    for i in 1:size(J11, 1) for j in 1:size(J11, 2)
]
ann12 = [
    attr(y=i-1, x=j-1, text=as_text(J12[i,j]), xref="x2", yref="y2", showarrow=false)
    for i in 1:size(J12, 1) for j in 1:size(J12, 2)
]
ann13 = [
    attr(y=i-1, x=j-1, text=as_text(J13[i,j]), xref="x3", yref="y3", showarrow=false)
    for i in 1:size(J13, 1) for j in 1:size(J13, 2)
]
ann14 = [
    attr(y=i-1, x=j-1, text=as_text(J14[i,j]), xref="x4", yref="y4", showarrow=false)
    for i in 1:size(J14, 1) for j in 1:size(J14, 2)
]
ann21 = [
    attr(y=i-1, x=j-1, text=as_text(J21[i,j]),  xref="x5", yref="y5", showarrow=false)
    for i in 1:size(J21, 1) for j in 1:size(J21, 2)
]
ann22 = [
    attr(y=i-1, x=j-1, text=as_text(J22[i,j]),  xref="x6", yref="y6", showarrow=false)
    for i in 1:size(J22, 1) for j in 1:size(J22, 2)
]
ann23 = [
    attr(y=i-1, x=j-1, text=as_text(J23[i,j]),  xref="x7", yref="y7", showarrow=false)
    for i in 1:size(J23, 1) for j in 1:size(J23, 2)
]
ann24 = [
    attr(y=i-1, x=j-1, text=as_text(J24[i,j]),  xref="x8", yref="y8", showarrow=false)
    for i in 1:size(J24, 1) for j in 1:size(J24, 2)
]
#
annotations = vcat(ann11, ann12, ann13, ann14, ann21, ann22, ann23, ann24,)

sp_titles = reshape([
    "bp / y",
    "bp / %",
    "bp / bp",
    "bp / %",
    #
    "% / y",
    "% / %",
    "% / bp",
    "% / %",
    ], (2,4)
)

p = make_subplots(
    rows=2,
    cols=4,
    subplot_titles = sp_titles,
    vertical_spacing = 0.15,
    horizontal_spacing = 0.15,
)

add_trace!(p, hm11, row=1, col=1)
add_trace!(p, hm21, row=2, col=1)
add_trace!(p, hm12, row=1, col=2)
add_trace!(p, hm22, row=2, col=2)
add_trace!(p, hm13, row=1, col=3)
add_trace!(p, hm23, row=2, col=3)
add_trace!(p, hm14, row=1, col=4)
add_trace!(p, hm24, row=2, col=4)

relayout!(p,
    height = 800,
    width = 1200,
    title = "Refrence Rate Sensitivities: Volatility (top) and Correlations (bottom)",
    yaxis_autorange = "reversed",
    yaxis2_autorange = "reversed",
    yaxis3_autorange = "reversed",
    yaxis4_autorange = "reversed",
    yaxis5_autorange = "reversed",
    yaxis6_autorange = "reversed",
    yaxis7_autorange = "reversed",
    yaxis8_autorange = "reversed",
)
append!(p.plot.layout.annotations, annotations)

display(p)